In [4]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os.path as osp
import cv2

In [6]:
class MNISTDataset(Dataset):
    def __init__(self, root, transform = None, preload = False):
        self.images = None
        self.labels = None
        self.fileNames = []
        self.root = root
        self.transform = transform
        
        for i in range(10):
            for filename in glob.glob(osp.join(root, str(i), "*.png")):
                self.fileNames.append((filename, i))

        self.len = len(self.fileNames)
        
        if preload: self.__preload()
        
    def __preload(self):
        self.labels = []
        self.images = []
        for fn, label in self.fileNames:
            self.images.append(cv2.imread(fn))
            self.labels.append(label)

    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        label = None
        image = None
        if self.images is not None:
            image = self.images[index]
            label = self.labels[index]
        else:
            fn, label = self.fileNames[index]
            image = cv2.imread(fn)
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [7]:
trainset = MNISTDataset("./mnist_png/training", 
                        transform = lambda x: transforms.ToTensor(cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)), 
                        preload = True)

testset = MNISTDataset("./mnist_png/testing", 
                        transform = lambda x: transforms.ToTensor(cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)), 
                        preload = True)

In [10]:
trainsetLoader = DataLoader(trainset, batch_size = 64, shuffle = True, num_workers = 1)
testsetLoader = DataLoader(testset, batch_size = 1000, shuffle = False, num_workers = 1)
print(f"train set length = {len(trainset)}")
print(f"test set length = {len(testset)}")

train set length = 60000
test set length = 10000


In [11]:
import matplotlib.pyplot as plt
def imshow(img):
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)))

dataiter = iter(trainsetLoader)
image, label = dataiter.next()

imshow(torchvision.utils.make_grid(images))

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/tuantran/anaconda3/envs/p37env/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/tuantran/anaconda3/envs/p37env/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/tuantran/anaconda3/envs/p37env/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-6-a14ce327e00a>", line 38, in __getitem__
    image = self.transform(image)
  File "<ipython-input-7-b0f56886b3f2>", line 2, in <lambda>
    transform = lambda x: transforms.ToTensor(cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)),
TypeError: ToTensor() takes no arguments
